# RAG Proportion Analysis

Here we will evaluate the behavior of the collections when using 25, 50 and 100 examples as subset provided by the RAG to perform the re-ranking  
The goal is to undestand if concetration of retrieved examples can improve the datamodels training

In [1]:

import os
import random
import numpy as np
import torch

import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed = 42

torch.backends.cudnn.enabled = False
# NumPy
np.random.seed(seed)
random.seed(seed)
# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Number of GPUs available: 1
GPU 0: Tesla T4


## Loading data

In [ ]:
paths = ["rag_25", "rag_50", "rag_100"]
collections = []
for collection_path in paths:
    dir_path = f"{collection_path}/datamodels/collections/train"
    for file in os.listdir(dir_path):
        collection = pl.read_ipc(os.path.join(dir_path, file))
        collection.with_columns(
            pl.lit(f"{collection_path}").alias("rag_subset"),
            pl.lit("train").alias("split"),
        )
    collections.append(collection)

    dir_path = f"{collection_path}/datamodels/collections/test"
    for file in os.listdir(dir_path):
        collection = pl.read_ipc(os.path.join(dir_path, file))
        collection.with_columns(
            pl.lit(f"{collection_path}").alias("rag_subset"),
            pl.lit("train").alias("split"),
        )
    collections.append(collection)
